# Zillow Data via Quandl

In [16]:
import os
import sys
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from uszipcode import SearchEngine

# sys.path.append('C:Users/Rob/repos/labspt15-cityspire-g-ds')
# sys.path

# path = r'C:\Users\Rob\repos\\'
# path == r'V:\\'

In [17]:
# quandl.ApiConfig.api_key = 'rfzykh-NDsw72dmc4Ms4'

In [18]:
# data = quandl.get_table('MER/F1', compnumber="39102", paginate=True)

In [19]:
# quandl.ApiConfig.api_key = 'rfzykh-NDsw72dmc4Ms4'
# quandl.get_table('ZILLOW/DATA', indicator_id='ZSFH', region_id='99999')

## Data

In [20]:
data = pd.read_csv("V:\ZILLOW_DATA.csv")
data.head()

,indicator_id,region_id,date,value
0,ZATT,3101,1998-01-31,338849.0
1,ZATT,3101,1998-02-28,342993.0
2,ZATT,3101,1998-03-31,346763.0
3,ZATT,3101,1998-04-30,349356.0
4,ZATT,3101,1998-05-31,351981.0


In [21]:
len(data) #125.5 million lines!!

125528409

In [22]:
data['indicator_id'].unique()

array(['ZATT', 'ZSFH', 'ZALL', 'ZCON', 'ZABT', 'Z5BR', 'Z2BR', 'Z3BR',
       'Z1BR', 'Z4BR', 'ISAW', 'SSAW', 'IRAM', 'CRSM', 'CSSM', 'MSAM',
       'IRAW', 'MRAM', 'SSSW', 'LSSM', 'CSAW', 'ISSM', 'SASW', 'MRAW',
       'LRAM', 'ISSW', 'MSAW', 'LSAM', 'CRAM', 'ISAM', 'IRSM', 'SRSW',
       'LSAW', 'NRAW', 'NSAW', 'CRSW', 'SSAM', 'IRSW', 'NRAM', 'CRAW',
       'LRSM', 'SASM', 'SSSM', 'SAAM', 'LRAW', 'CSAM', 'SRAW', 'CSSW',
       'LRSW', 'SRSM', 'SAAW', 'LSSW', 'SRAM', 'NSAM', 'RSNA', 'RSSA'],
      dtype=object)

In [23]:
rental_data = data[data['indicator_id'].isin(['RSSA', 'RSNA'])]

In [24]:
print(len(rental_data))
print(rental_data.shape)
rental_data.head()

535891
(535891, 4)


,indicator_id,region_id,date,value
1586389,RSNA,102001,2020-06-30,1777.0
1586392,RSNA,394913,2020-06-30,2811.0
1586396,RSNA,753899,2020-06-30,2625.0
1586400,RSNA,394463,2020-06-30,1804.0
1586403,RSNA,394514,2020-06-30,1611.0


In [25]:
print('average rent: ', round(sum(rental_data['value']) / len(rental_data)))

average rent:  1686


## Indicators

In [26]:
indicators = pd.read_csv('V:\ZILLOW_INDICATORS.csv')
indicators.head()

,indicator_id,indicator,category
0,ZATT,ZHVI All Homes- Top Tier Time Series ($),Home values
1,ZSFH,ZHVI Single-Family Homes Time Series ($),Home values
2,ZALL,"ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)",Home values
3,ZCON,ZHVI Condo/Co-op Time Series ($),Home values
4,ZABT,ZHVI All Homes- Bottom Tier Time Series ($),Home values


In [27]:
indicators['indicator_id'].unique()

array(['ZATT', 'ZSFH', 'ZALL', 'ZCON', 'ZABT', 'Z5BR', 'Z2BR', 'Z3BR',
       'Z1BR', 'LSAW', 'NRAW', 'NSAW', 'CRSW', 'SSAM', 'IRSW', 'NRAM',
       'CRAW', 'LRSM', 'SASM', 'SSSM', 'SAAM', 'LRAW', 'CSAM', 'SRAW',
       'CSSW', 'LRSW', 'SRSM', 'SAAW', 'LSSW', 'SRAM', 'NSAM', 'RSNA',
       'RSSA', 'Z4BR', 'ISAW', 'SSAW', 'IRAM', 'CRSM', 'CSSM', 'MSAM',
       'IRAW', 'MRAM', 'SSSW', 'LSSM', 'CSAW', 'ISSM', 'SASW', 'MRAW',
       'LRAM', 'ISSW', 'MSAW', 'LSAM', 'CRAM', 'ISAM', 'IRSM', 'SRSW'],
      dtype=object)

In [28]:
indicators

,indicator_id,indicator,category
0,ZATT,ZHVI All Homes- Top Tier Time Series ($),Home values
1,ZSFH,ZHVI Single-Family Homes Time Series ($),Home values
2,ZALL,"ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)",Home values
3,ZCON,ZHVI Condo/Co-op Time Series ($),Home values
4,ZABT,ZHVI All Homes- Bottom Tier Time Series ($),Home values
5,Z5BR,ZHVI 5+ Bedroom Time Series ($),Home values
6,Z2BR,ZHVI 2-Bedroom Time Series ($),Home values
7,Z3BR,ZHVI 3-Bedroom Time Series ($),Home values
8,Z1BR,ZHVI 1-Bedroom Time Series ($),Home values
9,LSAW,"Median List Price (Smooth, All Homes, Weekly V...",Inventory and sales


In [29]:
indicators['category'].unique()

array(['Home values', 'Inventory and sales', 'Rentals'], dtype=object)

In [30]:
rentals = indicators[indicators['category'] == 'Rentals']
rentals.head()

,indicator_id,indicator,category
31,RSNA,ZORI (Smoothed): All Homes Plus Multifamily Ti...,Rentals
32,RSSA,"ZORI (Smoothed, Seasonally Adjusted): All Home...",Rentals


In [31]:
for item in rentals['indicator_id']:
    print(item)

RSNA
RSSA


## Regions

In [32]:
regions = pd.read_csv('V:\ZILLOW_REGIONS.csv')
regions = regions.sort_values(by=['region_id'])
regions.head(20)

,region_id,region_type,region
1150,3,state,Alaska; AK
453,4,state,Alabama; AL
515,6,state,Arkansas; AR
30,8,state,Arizona; AZ
10,9,state,California; CA
2555,10,state,Colorado; CO
349,11,state,Connecticut; CT
1243,12,state,District of Columbia; DC
1149,13,state,Delaware; DE
11,14,state,Florida; FL


In [33]:
len(regions)

78730

In [34]:
regions['region_type'].unique()

array(['state', 'county', 'city', 'neigh', 'zip', 'metro'], dtype=object)

In [35]:
len(regions['region_type'] == 'metro')

78730

In [36]:
metro = regions[regions['region_type'] == 'metro']
metro[:50]

,region_id,region_type,region
5176,102001,metro,United States
13277,394297,metro,"Aberdeen, SD; SD"
13668,394298,metro,"Aberdeen, WA; WA"
1213,394299,metro,"Abilene, TX; TX"
4449,394300,metro,"Ada, OK; OK"
3469,394302,metro,"Adrian, MI; MI"
1237,394304,metro,"Akron, OH"
5831,394305,metro,"Alamogordo, NM; NM"
2423,394306,metro,"Albany, GA; GA"
2899,394307,metro,"Albany, OR; OR"


# Data.World Population Data Set

In [37]:
population = pd.read_excel('https://query.data.world/s/gewikohoj4s2gqv4bowlzpdxeu4jzl', usecols=['State', 'City', 'Population'])
population.head()

,State,City,Population
0,Alabama,Abernant,7599
1,Alabama,Alabaster,31545
2,Alabama,Albertville,21458
3,Alabama,Alexander City,14849
4,Alabama,Andalusia,9081


In [38]:
len(population)

6889

In [39]:
population['City']

0             Abernant
1            Alabaster
2          Albertville
3       Alexander City
4            Andalusia
             ...      
6884            Lander
6885        Torrington
6886           Douglas
6887            Powell
6888    Bridger Valley
Name: City, Length: 6889, dtype: object

In [40]:
cities = population[population['Population'] > 80000]
len(cities)

500

In [41]:
len(cities['State'].unique())

47

In [42]:
cities['State'].unique()

array(['Alabama', 'Alaska', 'Arkansas', 'Arizona', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'DC', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maryland', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska',
       'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Virginia', 'Washington',
       'Wisconsin'], dtype=object)

In [43]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 
'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 
'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 
'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 
'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire','New Jersey', 
'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 
'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 
'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 
'West Virginia', 'Wisconsin', 'Wyoming']

In [44]:
missing = []

for state in states:
    if state not in list(cities['State'].unique()):
        missing.append(state)

print(missing)

['Maine', 'Vermont', 'West Virginia', 'Wyoming']


# Aggregating Data

In [45]:
# Checking if data has all the same sections
sorted(data['indicator_id'].unique()) == sorted(indicators['indicator_id'].unique())

True

In [46]:
rental_data = rental_data.sort_values(by=['region_id'])
rental_data = rental_data[rental_data['indicator_id'].isin(['RSSA', 'RSNA'])]
print(rental_data.shape)
rental_data.head()

(535891, 4)


,indicator_id,region_id,date,value
7851413,RSNA,58362,2020-09-30,1321.0
8747089,RSNA,58362,2020-05-31,1243.0
8747088,RSNA,58362,2020-06-30,1220.0
8747082,RSNA,58362,2020-07-31,1351.0
8753596,RSSA,58362,2020-07-31,1296.0


,indicator_id,region_id,date,value
7851413,RSNA,58362,2020-09-30,1321.0
8753679,RSSA,58362,2020-06-30,1251.0
8747088,RSNA,58362,2020-06-30,1220.0
8747082,RSNA,58362,2020-07-31,1351.0
8747089,RSNA,58362,2020-05-31,1243.0
...,...,...,...,...
51008783,RSSA,753924,2019-10-31,2105.0
1610766,RSSA,753924,2020-04-30,2093.0
51008768,RSSA,753924,2018-07-31,2057.0
51008757,RSSA,753924,2017-08-31,2029.0


In [47]:
rent_per_region = pd.merge(rental_data, regions, on='region_id')
rent_per_region.head()

,indicator_id,region_id,date,value,region_type,region
0,RSNA,58362,2020-09-30,1321.0,zip,01432; MA; Boston-Cambridge-Newton; Middlesex ...
1,RSNA,58362,2020-05-31,1243.0,zip,01432; MA; Boston-Cambridge-Newton; Middlesex ...
2,RSNA,58362,2020-06-30,1220.0,zip,01432; MA; Boston-Cambridge-Newton; Middlesex ...
3,RSNA,58362,2020-07-31,1351.0,zip,01432; MA; Boston-Cambridge-Newton; Middlesex ...
4,RSSA,58362,2020-07-31,1296.0,zip,01432; MA; Boston-Cambridge-Newton; Middlesex ...


In [48]:
len(rent_per_region['region'].unique())

4031

In [49]:
rent_per_region[[
    'zip', 'state', 'city', 'county', 'neighborhood'
    ]] = rent_per_region['region'].str.split(';', expand=True)

In [50]:
test = rent_per_region[::-1]
test = test[:100]
test.head(20)

,indicator_id,region_id,date,value,region_type,region,zip,state,city,county,neighborhood
535890,RSSA,753924,2017-04-30,2023.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535889,RSSA,753924,2019-12-31,2107.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535888,RSSA,753924,2020-03-31,2092.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535887,RSSA,753924,2020-04-30,2093.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535886,RSSA,753924,2019-10-31,2105.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535885,RSSA,753924,2019-09-30,2102.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535884,RSSA,753924,2019-08-31,2100.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535883,RSSA,753924,2019-07-31,2098.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535882,RSSA,753924,2019-06-30,2095.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None
535881,RSSA,753924,2019-05-31,2092.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",None,None,None,None


In [51]:
test[['city', 'state']] = test['region'].str.split(',', expand=True)
test.head()

,indicator_id,region_id,date,value,region_type,region,zip,state,city,county,neighborhood
535890,RSSA,753924,2017-04-30,2023.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",HI,Urban Honolulu,None,None
535889,RSSA,753924,2019-12-31,2107.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",HI,Urban Honolulu,None,None
535888,RSSA,753924,2020-03-31,2092.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",HI,Urban Honolulu,None,None
535887,RSSA,753924,2020-04-30,2093.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",HI,Urban Honolulu,None,None
535886,RSSA,753924,2019-10-31,2105.0,metro,"Urban Honolulu, HI","Urban Honolulu, HI",HI,Urban Honolulu,None,None


In [52]:
rent_per_region[['city', 'state']] = rent_per_region['region'].str.split(',', expand=True)

In [53]:
len(rent_per_region['state'].unique())

42

In [54]:
rent_per_region.isna().sum()

indicator_id         0
region_id            0
date                 0
value                0
region_type          0
region               0
zip                  0
state           517923
city                 0
county          380909
neighborhood    393657
dtype: int64

In [55]:
rent_per_region.shape

(535891, 11)

In [56]:
len(rent_per_region['city'].unique())

4031

In [57]:
rent_per_region = rent_per_region[rent_per_region['zip'].str.contains(r"^\d+$")]

In [58]:
rent_per_region.tail()

,indicator_id,region_id,date,value,region_type,region,zip,state,city,county,neighborhood
535370,RSNA,753844,2014-09-30,989.0,zip,29486,29486,None,29486,None,None
535371,RSNA,753844,2014-08-31,1020.0,zip,29486,29486,None,29486,None,None
535372,RSSA,753844,2014-08-31,997.0,zip,29486,29486,None,29486,None,None
535373,RSNA,753844,2014-12-31,1049.0,zip,29486,29486,None,29486,None,None
535374,RSNA,753844,2015-08-31,1093.0,zip,29486,29486,None,29486,None,None


In [59]:
rent_per_region[rent_per_region['city'].str.contains(r"^\d+$")]

,indicator_id,region_id,date,value,region_type,region,zip,state,city,county,neighborhood
756,RSNA,58449,2020-07-31,1635.0,zip,01610,01610,None,01610,None,None
757,RSNA,58449,2015-08-31,1049.0,zip,01610,01610,None,01610,None,None
758,RSSA,58449,2020-03-31,1338.0,zip,01610,01610,None,01610,None,None
759,RSSA,58449,2020-10-31,1370.0,zip,01610,01610,None,01610,None,None
760,RSNA,58449,2018-10-31,1365.0,zip,01610,01610,None,01610,None,None
...,...,...,...,...,...,...,...,...,...,...,...
535370,RSNA,753844,2014-09-30,989.0,zip,29486,29486,None,29486,None,None
535371,RSNA,753844,2014-08-31,1020.0,zip,29486,29486,None,29486,None,None
535372,RSSA,753844,2014-08-31,997.0,zip,29486,29486,None,29486,None,None
535373,RSNA,753844,2014-12-31,1049.0,zip,29486,29486,None,29486,None,None


## Zip Code doc

In [249]:
zips = pd.read_csv("V:\\uszips.csv")
zips = zips[['zip', 'lat', 'lng', 'city', 'state_id', 'state_name', 'population', 'density', 'county_name']]
zips.head()

,zip,lat,lng,city,state_id,state_name,population,density,county_name
0,601,18.18005,-66.75218,Adjuntas,PR,Puerto Rico,17113.0,102.7,Adjuntas
1,602,18.36074,-67.17519,Aguada,PR,Puerto Rico,37751.0,476.0,Aguada
2,603,18.45440,-67.12201,Aguadilla,PR,Puerto Rico,47081.0,574.9,Aguadilla
3,606,18.16721,-66.93828,Maricao,PR,Puerto Rico,6392.0,58.3,Maricao
4,610,18.29032,-67.12244,Anasco,PR,Puerto Rico,26686.0,286.9,Añasco


In [250]:
zips['state_id'].unique()

array(['PR', 'VI', 'MA', 'RI', 'NH', 'ME', 'VT', 'CT', 'NY', 'NJ', 'PA',
       'DE', 'DC', 'VA', 'MD', 'WV', 'NC', 'SC', 'GA', 'FL', 'AL', 'TN',
       'MS', 'KY', 'OH', 'IN', 'MI', 'IA', 'WI', 'MN', 'SD', 'ND', 'MT',
       'IL', 'MO', 'KS', 'NE', 'LA', 'AR', 'OK', 'TX', 'CO', 'WY', 'ID',
       'UT', 'AZ', 'NM', 'NV', 'CA', 'HI', 'AS', 'GU', 'MP', 'OR', 'WA',
       'AK'], dtype=object)

In [251]:
values = ['PR', 'VI', 'AS', 'GU', 'MP']

zips = zips[zips['state_id'].isin(values) == False]

In [252]:
len(zips['city'].unique())

17380

In [253]:
zips

,zip,lat,lng,city,state_id,state_name,population,density,county_name
142,1001,42.06259,-72.62589,Agawam,MA,Massachusetts,17312.0,581.0,Hampden
143,1002,42.37492,-72.46210,Amherst,MA,Massachusetts,30014.0,210.5,Hampshire
144,1003,42.39192,-72.52479,Amherst,MA,Massachusetts,11357.0,6164.3,Hampshire
145,1005,42.42017,-72.10615,Barre,MA,Massachusetts,5128.0,44.7,Worcester
146,1007,42.27875,-72.40036,Belchertown,MA,Massachusetts,15005.0,110.1,Hampshire
...,...,...,...,...,...,...,...,...,...
33116,99923,55.97796,-130.03671,Hyder,AK,Alaska,14.0,0.3,Prince of Wales-Hyder
33117,99925,55.55796,-132.97482,Klawock,AK,Alaska,908.0,6.3,Prince of Wales-Hyder
33118,99926,55.12617,-131.48928,Metlakatla,AK,Alaska,1654.0,4.8,Prince of Wales-Hyder
33119,99927,56.25100,-133.37572,Point Baker,AK,Alaska,0.0,0.0,Prince of Wales-Hyder


In [254]:
city_list = pd.read_csv('V:\\cityspire_cities.csv')
print(len(city_list))
city_list.head()

362


,Cities_List
0,"Akron, OH"
1,"Albany, GA"
2,"Albany, NY"
3,"Albany, OR"
4,"Albuquerque, NM"


In [255]:
len(city_list['Cities_List'].unique())

362

In [256]:
city_names = [city.split(",")[0] for city in city_list['Cities_List']]

In [257]:
len(city_names)

362

In [258]:
zips = zips[zips['city'].isin(city_names) == True]
zips

,zip,lat,lng,city,state_id,state_name,population,density,county_name
176,1062,42.32188,-72.69278,Florence,MA,Massachusetts,10364.0,220.4,Hampshire
192,1083,42.20382,-72.19495,Warren,MA,Massachusetts,3094.0,75.8,Worcester
205,1103,42.10400,-72.59210,Springfield,MA,Massachusetts,2332.0,1956.6,Hampden
206,1104,42.13404,-72.56679,Springfield,MA,Massachusetts,23740.0,1757.3,Hampden
207,1105,42.09917,-72.57899,Springfield,MA,Massachusetts,11841.0,3678.4,Hampden
...,...,...,...,...,...,...,...,...,...
33028,99712,64.99326,-146.14966,Fairbanks,AK,Alaska,14837.0,1.6,Fairbanks North Star
33050,99744,64.30627,-149.18016,Anderson,AK,Alaska,120.0,1.4,Denali
33081,99775,64.85849,-147.82472,Fairbanks,AK,Alaska,1180.0,3907.3,Fairbanks North Star
33094,99790,65.42127,-148.28799,Fairbanks,AK,Alaska,20.0,0.0,Yukon-Koyukuk


In [259]:
# Accounting for zip code error with dropped leading 0 (east cost, mostly)
zips['zip'] = zips['zip'].astype(str)
zips['zip'] = zips['zip'].str.zfill(5)

C:\Users\Rob\AppData\Local\Temp/ipykernel_11056/3169711894.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zips['zip'] = zips['zip'].astype(str)
C:\Users\Rob\AppData\Local\Temp/ipykernel_11056/3169711894.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zips['zip'] = zips['zip'].str.zfill(5)


In [262]:
zips

,zip,lat,lng,city,state_id,state_name,population,density,county_name
176,01062,42.32188,-72.69278,Florence,MA,Massachusetts,10364.0,220.4,Hampshire
192,01083,42.20382,-72.19495,Warren,MA,Massachusetts,3094.0,75.8,Worcester
205,01103,42.10400,-72.59210,Springfield,MA,Massachusetts,2332.0,1956.6,Hampden
206,01104,42.13404,-72.56679,Springfield,MA,Massachusetts,23740.0,1757.3,Hampden
207,01105,42.09917,-72.57899,Springfield,MA,Massachusetts,11841.0,3678.4,Hampden
...,...,...,...,...,...,...,...,...,...
33028,99712,64.99326,-146.14966,Fairbanks,AK,Alaska,14837.0,1.6,Fairbanks North Star
33050,99744,64.30627,-149.18016,Anderson,AK,Alaska,120.0,1.4,Denali
33081,99775,64.85849,-147.82472,Fairbanks,AK,Alaska,1180.0,3907.3,Fairbanks North Star
33094,99790,65.42127,-148.28799,Fairbanks,AK,Alaska,20.0,0.0,Yukon-Koyukuk


In [263]:
zips.to_csv(r'V:\cities_with_zips.csv', index = False)

## Combing datasets on zip codes

In [264]:
rent_per_region.head()

,indicator_id,region_id,date,value,region_type,region,zip,state,city,county,neighborhood
0,RSNA,58362,2020-09-30,1321.0,zip,01432; MA; Boston-Cambridge-Newton; Middlesex ...,01432,None,01432; MA; Boston-Cambridge-Newton; Middlesex ...,Middlesex County,Ayer
1,RSNA,58362,2020-05-31,1243.0,zip,01432; MA; Boston-Cambridge-Newton; Middlesex ...,01432,None,01432; MA; Boston-Cambridge-Newton; Middlesex ...,Middlesex County,Ayer
2,RSNA,58362,2020-06-30,1220.0,zip,01432; MA; Boston-Cambridge-Newton; Middlesex ...,01432,None,01432; MA; Boston-Cambridge-Newton; Middlesex ...,Middlesex County,Ayer
3,RSNA,58362,2020-07-31,1351.0,zip,01432; MA; Boston-Cambridge-Newton; Middlesex ...,01432,None,01432; MA; Boston-Cambridge-Newton; Middlesex ...,Middlesex County,Ayer
4,RSSA,58362,2020-07-31,1296.0,zip,01432; MA; Boston-Cambridge-Newton; Middlesex ...,01432,None,01432; MA; Boston-Cambridge-Newton; Middlesex ...,Middlesex County,Ayer


In [266]:
rent_per_region = rent_per_region[['indicator_id', 'date', 'value', 'region', 'zip']]
rent_per_region.head()

,indicator_id,date,value,region,zip
0,RSNA,2020-09-30,1321.0,01432; MA; Boston-Cambridge-Newton; Middlesex ...,01432
1,RSNA,2020-05-31,1243.0,01432; MA; Boston-Cambridge-Newton; Middlesex ...,01432
2,RSNA,2020-06-30,1220.0,01432; MA; Boston-Cambridge-Newton; Middlesex ...,01432
3,RSNA,2020-07-31,1351.0,01432; MA; Boston-Cambridge-Newton; Middlesex ...,01432
4,RSSA,2020-07-31,1296.0,01432; MA; Boston-Cambridge-Newton; Middlesex ...,01432


In [265]:
zips.head()

,zip,lat,lng,city,state_id,state_name,population,density,county_name
176,01062,42.32188,-72.69278,Florence,MA,Massachusetts,10364.0,220.4,Hampshire
192,01083,42.20382,-72.19495,Warren,MA,Massachusetts,3094.0,75.8,Worcester
205,01103,42.10400,-72.59210,Springfield,MA,Massachusetts,2332.0,1956.6,Hampden
206,01104,42.13404,-72.56679,Springfield,MA,Massachusetts,23740.0,1757.3,Hampden
207,01105,42.09917,-72.57899,Springfield,MA,Massachusetts,11841.0,3678.4,Hampden


In [267]:
rent_data = rent_per_region.merge(zips, on="zip", how = 'inner')
rent_data.head()

,indicator_id,date,value,region,zip,lat,lng,city,state_id,state_name,population,density,county_name
0,RSNA,2020-08-31,1375.0,01603; MA; Worcester; Worcester County,01603,42.24407,-71.84445,Worcester,MA,Massachusetts,19731.0,1688.4,Worcester
1,RSNA,2020-07-31,1402.0,01603; MA; Worcester; Worcester County,01603,42.24407,-71.84445,Worcester,MA,Massachusetts,19731.0,1688.4,Worcester
2,RSNA,2020-06-30,1439.0,01603; MA; Worcester; Worcester County,01603,42.24407,-71.84445,Worcester,MA,Massachusetts,19731.0,1688.4,Worcester
3,RSNA,2020-11-30,1465.0,01603; MA; Worcester; Worcester County,01603,42.24407,-71.84445,Worcester,MA,Massachusetts,19731.0,1688.4,Worcester
4,RSNA,2020-10-31,1566.0,01603; MA; Worcester; Worcester County,01603,42.24407,-71.84445,Worcester,MA,Massachusetts,19731.0,1688.4,Worcester


In [268]:
rent_data.shape

(213209, 13)